In [4]:
import sys
import os
import torch
import matplotlib.pyplot as plt

sys.path.append(os.path.abspath(sys.path[0] + '/../../'))
__package__ = "deep_view_syn.notebook"
torch.cuda.set_device(2)
print("Set CUDA:%d as current device." % torch.cuda.current_device())
torch.autograd.set_grad_enabled(False)

from ..data.spherical_view_syn import *
from ..configs.spherical_view_syn import SphericalViewSynConfig
from ..my import netio
from ..my import util
from ..my import device
from ..my import view
from ..my.gen_final import GenFinal


def load_net(path):
    config = SphericalViewSynConfig()
    config.from_id(path[:-4])
    config.SAMPLE_PARAMS['perturb_sample'] = False
    #config.print()
    net = config.create_net().to(device.GetDevice())
    netio.LoadNet(path, net)
    return net


def find_file(prefix):
    for path in os.listdir():
        if path.startswith(prefix):
            return path
    return None


def load_views(data_desc_file) -> Tuple[view.Trans, torch.Tensor]:
    with open(data_desc_file, 'r', encoding='utf-8') as file:
        lines = file.readlines()
        n = len(lines) // 7
        gazes = torch.empty(n * 2, 3)
        views = torch.empty(n * 2, 4, 4)
        view_idx = 0
        for i in range(0, len(lines), 7):
            gazes[view_idx * 2] = torch.tensor([
                float(str) for str in lines[i + 1].split(',')
            ])
            gazes[view_idx * 2 + 1] = torch.tensor([
                float(str) for str in lines[i + 2].split(',')
            ])
            views[view_idx * 2] = torch.tensor([
                float(str) for str in lines[i + 3].split(',')
            ]).view(4, 4)
            views[view_idx * 2 + 1] = torch.tensor([
                float(str) for str in lines[i + 4].split(',')
            ]).view(4, 4)
            view_idx += 1
        gazes = gazes.to(device.GetDevice())
        views = views.to(device.GetDevice())
    return view.Trans(views[:, :3, 3], views[:, :3, :3]), gazes

fov_list = [20, 45, 110]
res_list = [(128, 128), (256, 256), (256, 230)]  # (192,256)]
res_full = (1600, 1440)


Set CUDA:2 as current device.


In [5]:
os.chdir(sys.path[0] + '/../data/__0_user_study/us_gas_all_in_one')
#os.chdir(sys.path[0] + '/../data/__0_user_study/us_mc_all_in_one')
#os.chdir(sys.path[0] + '/../data/bedroom_all_in_one')
#os.chdir(sys.path[0] + '/../data/lobby_all_in_one')
#os.chdir(sys.path[0] + '/../data/gallery_all_in_one')
print('Change working directory to ', os.getcwd())

fovea_net = load_net(find_file('fovea'))
periph_net = load_net(find_file('periph'))

# Load Dataset
views, gazes = load_views('hmd.csv')
print('Dataset loaded.')
print('views:', views.size())

gen = GenFinal(fov_list, res_list, res_full, fovea_net, periph_net,
               device=device.GetDevice())
gaze_centers = gen.full_cam.proj(gazes, center_as_origin=True)


Change working directory to  /home/dengnc/deep_view_syn/data/__0_user_study/us_gas_all_in_one
Load net from fovea@nmsl-rgb_e10_fc128x4_d1-50_s32.pth ...
Load net from periph_rgb@msl-rgb_e10_fc96x4_d1.00-50.00_s16.pth ...
Dataset loaded.
views: [1456]


In [3]:
for view_idx in range(gaze_centers.size(0) / 2):
    left_center = (gaze_centers[view_idx * 2][0].item(),
                   gaze_centers[view_idx * 2][1].item())
    right_center = (gaze_centers[view_idx * 2 + 1][0].item(),
                    gaze_centers[view_idx * 2 + 1][1].item())
    left_view = views.get(view_idx * 2)
    right_view = views.get(view_idx * 2 + 1)
    mono_trans = view.Trans((left_view.t + right_view.t) / 2, left_view.r)
    left_images = gen.gen(left_center, left_view, mono_trans=mono_trans)
    right_images = gen.gen(right_center, right_view, mono_trans=mono_trans)
    
    util.CreateDirIfNeed('output/video_frames/hmd2')
    util.WriteImageTensor(torch.cat([left_images['blended'], right_images['blended']], -1),
                          'output/video_frames/hmd2/view%04d.png' % view_idx)
    print('Frame %d saved' % view_idx)


Frame 0 saved
Frame 1 saved
Frame 2 saved
Frame 3 saved
Frame 4 saved
Frame 5 saved
Frame 6 saved
Frame 7 saved
Frame 8 saved
Frame 9 saved
Frame 10 saved
Frame 11 saved
Frame 12 saved
Frame 13 saved
Frame 14 saved
Frame 15 saved
Frame 16 saved
Frame 17 saved
Frame 18 saved
Frame 19 saved
Frame 20 saved
Frame 21 saved
Frame 22 saved
Frame 23 saved
Frame 24 saved
Frame 25 saved
Frame 26 saved
Frame 27 saved
Frame 28 saved
Frame 29 saved
Frame 30 saved
Frame 31 saved
Frame 32 saved
Frame 33 saved
Frame 34 saved
Frame 35 saved
Frame 36 saved
Frame 37 saved
Frame 38 saved
Frame 39 saved
Frame 40 saved
Frame 41 saved
Frame 42 saved
Frame 43 saved
Frame 44 saved
Frame 45 saved
Frame 46 saved
Frame 47 saved
Frame 48 saved
Frame 49 saved
Frame 50 saved
Frame 51 saved
Frame 52 saved
Frame 53 saved
Frame 54 saved
Frame 55 saved
Frame 56 saved
Frame 57 saved
Frame 58 saved
Frame 59 saved
Frame 60 saved
Frame 61 saved
Frame 62 saved
Frame 63 saved
Frame 64 saved
Frame 65 saved
Frame 66 saved
Frame

IndexError: index 1850 is out of bounds for dimension 0 with size 1850